In [86]:
import pandas as pd
import numpy as np
from mlforecast import MLForecast
from lightgbm import LGBMRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import root_mean_squared_error

##### Dataset "base"

Para generar el dataset base: Ejecutar este notebook

./generadores/generador_dataset_sellin.ipynb


In [87]:
df = pd.read_csv('../../data/preprocessed/base.csv', sep=',')
df.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,stock_final
0,201701,10234,20524,0,2,0.05300,0.05300,HC,VAJILLA,Cristalino,Importado,500.0,NaN
1,201701,10032,20524,0,1,0.13628,0.13628,HC,VAJILLA,Cristalino,Importado,500.0,NaN
2,201701,10217,20524,0,1,0.03028,0.03028,HC,VAJILLA,Cristalino,Importado,500.0,NaN
3,201701,10125,20524,0,1,0.02271,0.02271,HC,VAJILLA,Cristalino,Importado,500.0,NaN
4,201701,10012,20524,0,11,1.54452,1.54452,HC,VAJILLA,Cristalino,Importado,500.0,NaN


In [88]:
dfg = df.groupby(['periodo', 'product_id']).agg({'tn': 'sum'}).reset_index()
dfg.head()

,periodo,product_id,tn
0,201701,20001,934.77222
1,201701,20002,550.15707
2,201701,20003,1063.45835
3,201701,20004,555.91614
4,201701,20005,494.27011


In [89]:
dfg['periodo_dt'] = pd.to_datetime(dfg['periodo'].astype(str), format='%Y%m')
dfg.rename(columns={'tn': 'y', 'product_id':'unique_id', 'periodo_dt': 'ds'}, inplace=True)
dfg.drop(columns=['periodo'], inplace=True)
dfg

,unique_id,y,ds
0,20001,934.77222,2017-01-01
1,20002,550.15707,2017-01-01
2,20003,1063.45835,2017-01-01
3,20004,555.91614,2017-01-01
4,20005,494.27011,2017-01-01
...,...,...,...
31238,21265,0.05007,2019-12-01
31239,21266,0.05121,2019-12-01
31240,21267,0.01569,2019-12-01
31241,21271,0.00298,2019-12-01


In [90]:
productos_ok = pd.read_csv('../../data/raw/product_id_apredecir201912.csv', sep=',')
productos_ok.head()

,product_id
0,20001
1,20002
2,20003
3,20004
4,20005


In [91]:
dfg = dfg[dfg['unique_id'].isin(productos_ok['product_id'].unique())]

In [92]:
def get_productos_12_meses():
    df = pd.read_csv('../../data/raw/sell-in.csv', sep='\t')  # Cargar el dataset
    df = df.groupby(['periodo', 'product_id']).agg({'tn': 'sum'}).reset_index()  # Agrupar por periodo y producto

    # Paso 1: calcular la cantidad de registros por product_id
    conteo = df.groupby('product_id').size()

    # Paso 2: quedarte solo con los que tienen 36 registros
    productos_12_meses = conteo[conteo >= 36].index.tolist()

    

    return productos_12_meses

productos_12_meses = get_productos_12_meses()
dfg = dfg[dfg['unique_id'].isin(productos_12_meses)]

In [93]:
dfg = dfg.sort_values(['unique_id', 'ds'])

In [94]:
H = 2  # Horizonte de predicción: mes +2

fcst = MLForecast(
    models = [LGBMRegressor()],
    freq = 'MS',
    lags = [1, 2, 3],
    # lag_transforms = {
    #     #1: [np.log1p],
    #     1: ['mean'],
    #     3: ['std']
    # },
    date_features = ['month']
)


In [95]:
df = dfg.copy()
df = df[['unique_id', 'ds', 'y']].copy()
df


,unique_id,ds,y
0,20001,2017-01-01,934.77222
785,20001,2017-02-01,798.01620
1566,20001,2017-03-01,1303.35771
2352,20001,2017-04-01,1069.96130
3136,20001,2017-05-01,1502.20132
...,...,...,...
27468,21222,2019-08-01,0.04442
28400,21222,2019-09-01,0.08373
29347,21222,2019-10-01,0.06699
30287,21222,2019-11-01,0.02766


In [96]:
df.fillna(0, inplace=True)

In [97]:
df

,unique_id,ds,y
0,20001,2017-01-01,934.77222
785,20001,2017-02-01,798.01620
1566,20001,2017-03-01,1303.35771
2352,20001,2017-04-01,1069.96130
3136,20001,2017-05-01,1502.20132
...,...,...,...
27468,21222,2019-08-01,0.04442
28400,21222,2019-09-01,0.08373
29347,21222,2019-10-01,0.06699
30287,21222,2019-11-01,0.02766


In [98]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

# Diagnóstico inicial del dataset
print("=== DIAGNÓSTICO DEL DATASET ===")
print(f"Total de series únicas: {df['unique_id'].nunique()}")
print(f"Rango de fechas en df: {df['ds'].min()} a {df['ds'].max()}")

# Verificar la última fecha por serie
last_dates_per_series = df.groupby('unique_id')['ds'].max().reset_index()
print(f"\nDistribución de últimas fechas por serie:")
print(last_dates_per_series['ds'].value_counts().sort_index().tail(10))

# Identificar series que no llegan hasta el final
max_date_in_data = df['ds'].max()
series_with_max_date = last_dates_per_series[last_dates_per_series['ds'] == max_date_in_data]
print(f"\nSeries que llegan hasta {max_date_in_data}: {len(series_with_max_date)}")
print(f"Series que NO llegan hasta el final: {len(last_dates_per_series) - len(series_with_max_date)}")

# H = 2 porque queremos predecir 2 meses hacia adelante para llegar al mes+2
H = 2

unique_dates = df['ds'].sort_values().unique()
tscv = TimeSeriesSplit(n_splits=5)
errors = []

for fold, (train_idx, val_idx) in enumerate(tscv.split(unique_dates)):
    print(f"\n=== Fold {fold+1} ===")
    
    train_dates = unique_dates[train_idx]
    val_dates = unique_dates[val_idx]
    
    # Crear conjuntos de entrenamiento y validación
    df_train = df[df['ds'].isin(train_dates)].copy()
    df_val = df[df['ds'].isin(val_dates)].copy()
    
    print(f"Fechas entrenamiento: {train_dates.min()} a {train_dates.max()}")
    print(f"Fechas validación: {val_dates.min()} a {val_dates.max()}")
    print(f"Series en entrenamiento: {df_train['unique_id'].nunique()}")
    print(f"Series en validación: {df_val['unique_id'].nunique()}")
    
    # Verificar cuántas series llegan hasta el final del periodo de entrenamiento
    max_train_date = train_dates.max()
    last_dates_train = df_train.groupby('unique_id')['ds'].max().reset_index()
    series_complete_train = last_dates_train[last_dates_train['ds'] == max_train_date]
    
    print(f"Series completas en entrenamiento (hasta {max_train_date}): {len(series_complete_train)}")
    
    # Entrenar el modelo
    fcst.fit(df_train)
    
    # Hacer predicciones para H=2 períodos hacia adelante
    preds = fcst.predict(h=H)
    
    print(f"Predicciones generadas: {len(preds)}")
    print(f"Series con predicciones: {preds['unique_id'].nunique()}")
    
    # Obtener las fechas objetivo (mes+2) para cada serie
    last_train_dates = df_train.groupby('unique_id')['ds'].max().reset_index()
    
    print(f"Distribución de últimas fechas en entrenamiento:")
    print(last_train_dates['ds'].value_counts().sort_index().tail(5))
    
    # Crear las fechas objetivo específicas (mes+2)
    target_dates_list = []
    for _, row in last_train_dates.iterrows():
        unique_id = row['unique_id']
        last_date = row['ds']
        
        # Calcular la fecha objetivo (mes+2)
        if isinstance(last_date, pd.Timestamp):
            target_date = last_date + pd.DateOffset(months=2)
        else:
            # Si ds es string, convertir primero
            last_date_ts = pd.to_datetime(last_date)
            target_date = last_date_ts + pd.DateOffset(months=2)
        
        target_dates_list.append({
            'unique_id': unique_id, 
            'last_train_date': last_date,
            'target_date': target_date
        })
    
    target_dates_df = pd.DataFrame(target_dates_list)
    
    print(f"Ejemplo de fechas objetivo:")
    print(target_dates_df[['last_train_date', 'target_date']].head(3))
    
    # Verificar las fechas en las predicciones
    preds_by_series = preds.groupby('unique_id')['ds'].apply(list).reset_index()
    preds_by_series.columns = ['unique_id', 'pred_dates']
    
    print(f"Ejemplo de fechas predichas:")
    if len(preds_by_series) > 0:
        print(f"Serie {preds_by_series.iloc[0]['unique_id']}: {preds_by_series.iloc[0]['pred_dates']}")
    
    # Filtrar predicciones para obtener solo las del mes+2
    # Esto sería la segunda predicción (índice 1) para cada serie
    preds_month2 = preds[preds.groupby('unique_id').cumcount() == 1].copy()
    
    print(f"Predicciones mes+2 extraídas: {len(preds_month2)}")
    
    # Hacer merge con las fechas objetivo
    preds_month2 = preds_month2.merge(
        target_dates_df, 
        on='unique_id', 
        how='inner'
    )
    
    # Verificar que ds coincida con target_date (con tolerancia)
    preds_month2['date_match'] = preds_month2['ds'] == preds_month2['target_date']
    
    print(f"Predicciones que coinciden con fecha objetivo: {preds_month2['date_match'].sum()}")
    
    if not preds_month2['date_match'].all():
        print("Algunas fechas no coinciden exactamente:")
        mismatches = preds_month2[~preds_month2['date_match']]
        print(f"Ejemplos de desajustes ({len(mismatches)} casos):")
        for _, row in mismatches.head(3).iterrows():
            print(f"  Serie {row['unique_id']}: pred={row['ds']}, objetivo={row['target_date']}")
    
    # Filtrar solo las que coinciden exactamente
    preds_month2_final = preds_month2[preds_month2['date_match']].copy()
    
    print(f"Predicciones mes+2 finales: {len(preds_month2_final)}")
    
    # Buscar los datos reales correspondientes en el conjunto de validación
    merged = preds_month2_final.merge(
        df_val, 
        on=['unique_id', 'ds'], 
        how='inner'
    )
    
    print(f"Coincidencias para evaluación: {len(merged)}")
    
    if not merged.empty:
        # Obtener el nombre de la columna de predicción
        pred_cols = [col for col in merged.columns if col not in ['unique_id', 'ds', 'y', 'target_date', 'last_train_date', 'date_match']]
        
        if pred_cols:
            pred_col = pred_cols[0]  # Tomar la primera columna de predicción
            rmse = np.sqrt(mean_squared_error(merged['y'], merged[pred_col]))
            
            print(f"RMSE (mes+2): {rmse:.4f}")
            print(f"Usando columna de predicción: {pred_col}")
            errors.append(rmse)
        else:
            print("No se encontró columna de predicción válida")
            print("Columnas disponibles:", merged.columns.tolist())
            errors.append(np.nan)
    else:
        print("No hay coincidencias entre predicciones mes+2 y datos reales.")
        errors.append(np.nan)

# Calcular estadísticas finales
valid_errors = [e for e in errors if not np.isnan(e)]
if valid_errors:
    print(f"\n=== RESULTADOS FINALES ===")
    print(f"RMSE promedio (mes+2): {np.mean(valid_errors):.4f}")
    print(f"Desviación estándar RMSE: {np.std(valid_errors):.4f}")
    print(f"Folds válidos: {len(valid_errors)}/{len(errors)}")
    print(f"Rango RMSE: {np.min(valid_errors):.4f} - {np.max(valid_errors):.4f}")
else:
    print("\nNo se pudieron calcular métricas válidas en ningún fold.")

=== DIAGNÓSTICO DEL DATASET ===
Total de series únicas: 478
Rango de fechas en df: 2017-01-01 00:00:00 a 2019-12-01 00:00:00

Distribución de últimas fechas por serie:
ds
2019-12-01    478
Name: count, dtype: int64

Series que llegan hasta 2019-12-01 00:00:00: 478
Series que NO llegan hasta el final: 0

=== Fold 1 ===
Fechas entrenamiento: 2017-01-01 00:00:00 a 2017-06-01 00:00:00
Fechas validación: 2017-07-01 00:00:00 a 2017-12-01 00:00:00
Series en entrenamiento: 478
Series en validación: 478
Series completas en entrenamiento (hasta 2017-06-01 00:00:00): 478
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0,010675 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 769
[LightGBM] [Info] Number of data points in the train set: 1434, number of used features: 4
[LightGBM] [Info] Start training from score 70,767130
Predicciones generadas: 956
Series con predicciones: 478
Distribución de últimas fechas en en

In [99]:
merged

,unique_id,ds,LGBMRegressor,last_train_date,target_date,date_match,y
0,20001,2019-08-01,1479.494232,2019-06-01,2019-08-01,True,1261.34529
1,20002,2019-08-01,1277.955467,2019-06-01,2019-08-01,True,813.78215
2,20003,2019-08-01,817.735351,2019-06-01,2019-08-01,True,635.59563
3,20004,2019-08-01,771.393801,2019-06-01,2019-08-01,True,482.13372
4,20005,2019-08-01,833.603486,2019-06-01,2019-08-01,True,536.66800
...,...,...,...,...,...,...,...
473,21182,2019-08-01,1.496975,2019-06-01,2019-08-01,True,0.14046
474,21192,2019-08-01,1.496975,2019-06-01,2019-08-01,True,0.02840
475,21202,2019-08-01,1.496975,2019-06-01,2019-08-01,True,0.08585
476,21218,2019-08-01,1.496975,2019-06-01,2019-08-01,True,0.04513


In [100]:
print("Shape:", df.shape)
print("Columnas:", df.columns.tolist())
print("Tipos:", df.dtypes)

Shape: (17208, 3)
Columnas: ['unique_id', 'ds', 'y']
Tipos: unique_id             int64
ds           datetime64[ns]
y                   float64
dtype: object


In [101]:
print(df['unique_id'].nunique())              # cuántas series hay
print(df.groupby('unique_id').size().min())   # cuál es la más corta

478
36


In [102]:
def promedio_12_meses_780p():
    
    df = pd.read_csv("../entregable/datasets/periodo_x_producto_con_target.csv", sep=',', encoding='utf-8')
    df = df[df['periodo'] >= 201901]  # Filtrar desde 201901
    
    productos_ok = pd.read_csv("https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/product_id_apredecir201912.txt", sep="\t")

    df = df.merge(productos_ok, on='product_id', how='inner')
    
    df = df.groupby('product_id').agg({'tn': 'mean'}).reset_index()
    
    return df

promedios = promedio_12_meses_780p()

In [103]:
merged = merged.merge(
    promedios, 
    left_on='unique_id', 
    right_on='product_id', 
    how='outer'
)
merged

,unique_id,ds,LGBMRegressor,last_train_date,target_date,date_match,y,product_id,tn
0,20001.0,2019-08-01,1479.494232,2019-06-01,2019-08-01,True,1261.34529,20001,1454.732720
1,20002.0,2019-08-01,1277.955467,2019-06-01,2019-08-01,True,813.78215,20002,1175.437142
2,20003.0,2019-08-01,817.735351,2019-06-01,2019-08-01,True,635.59563,20003,784.976407
3,20004.0,2019-08-01,771.393801,2019-06-01,2019-08-01,True,482.13372,20004,627.215328
4,20005.0,2019-08-01,833.603486,2019-06-01,2019-08-01,True,536.66800,20005,668.270104
...,...,...,...,...,...,...,...,...,...
775,NaN,NaT,NaN,NaT,NaT,NaN,NaN,21263,0.029993
776,NaN,NaT,NaN,NaT,NaT,NaN,NaN,21265,0.089541
777,NaN,NaT,NaN,NaT,NaT,NaN,NaN,21266,0.094659
778,NaN,NaT,NaN,NaT,NaT,NaN,NaN,21267,0.092835


In [104]:
merged['LGBMRegressor'].fillna(merged['tn'], inplace=True)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_19912\3061097598.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged['LGBMRegressor'].fillna(merged['tn'], inplace=True)


In [105]:
merged[['product_id', 'LGBMRegressor']].to_csv("./outputs/forecast.csv", sep=',', index=False)